# Calculator and Visualizer for DSA-2000 Capabilities

## Casey Law, claw@astro.caltech.edu

Written in support of DSA-2000 science workshop (January 2022). Reproduces work by Steve Myers for VLASS and DSA-2000 survey design.

In [1]:
%matplotlib inline

import pylab as pl
import numpy as np
import healpy

## Set telescope parameters as a function of band/config

In [2]:
# keys defined for configurations and bands separately.

na = {'DFACS': 2000, 'VLASS': 27}   # number of antennas
freq = {'DSA': 1.35e9, 'VLAL': 1.5e9, 'VLAS': 3.0e9}  # band center
fov = {'DSA': 60*3.06,'VLAL': 30., 'VLAS': 15.}    # fwhm field of view (theta_pb) in arcminutes
bw_max = {'DSA': 0.65*1.3e9, 'VLAL': 0.6e9, 'VLAS': 1.5e9}    # bandwidth (minus RFI)
eta = {'DSA': 1.0, 'VLAL': 0.92, 'VLAS': 0.92}   # correlator efficiency
bmax = {'DSAW': 15e3, 'VLAA': 36.4e3, 'VLAB': 11.1e3, 'VLAC': 3.4e3, 'VLAD': 1.03e3}    # longest baselines in meter
sefd = {'DSA': 5020, 'VLAL': 510, 'VLAS': 370}    # array SEFD fixed to match exposure calculator int time to 100 microJy.
# VLA sefd values appear high, because we assume 27 ants, whereas exposure calculator uses 25 ants. result is the same in both cases.
surveyspecs = {'DFACS': [3e4, 2.04e-6, 'DSA', 1.25, 1.0], 'VLASS': [3.3885e4, 122e-6, 'VLAS', 1.17, 1.09], }
#               'EMU': []}
#'COSMOS': [2., 1.5e-6, 'VLAS', 1.25, 1.0]

## Define functions


In [3]:
resolution = lambda freq, bmax: 3600*np.degrees(3e8/float(freq)/bmax) # resolution in asec for freq in Hz, bmax in meters
sensitivity = lambda sefd, dt, bw, eta, nbl: sefd/(eta*np.sqrt(nbl*2 * dt * bw * 2))    # sefd in Jy, dt in s, bw in Hz, assumes std correlator eficiency and 2 pols
tobs = lambda sensitivity, sefd, na, bw, eta: (sefd/(sensitivity*eta))**2/(na*(na-1)*bw*2)    # obsering time in s (inverse of sensitivity eqn)
surveyspeed = lambda fov, tobs: 0.5665 * fov**2/tobs         # ss in deg2/hr, fov in amin, tobs in s
f_beam = lambda rad, fwhm: np.exp(-4*np.log(2)*(rad/fwhm)**2)

datarate_vis = lambda na, nch, npol, tsamp: (na*(na-1)/2)*nch*npol*8/tsamp/1024**2  # MB/s for tsamp in s
datarate_vis_vla = lambda tsamp, nspw: 45*(nspw*64*4/16384.) / tsamp # scaling taken from nrao oss, minimal nchan, full pol

In [4]:
ess_nu = lambda fov, tobs, nu, alpha: surveyspeed(fov, tobs) * nu**(2*alpha)
ess_flux = lambda fov, tobs, sensitivity, gamma: surveyspeed(fov, tobs) * sensitivity**(gamma+2)
ess_nuflux = lambda fov, tobs, nu, alpha, sensitivity, gamma: surveyspeed(fov, tobs) * sensitivity**(gamma+2) * nu**(2*alpha)
surveytime = lambda band, sensitivity, na, area, overhead: overhead * area/surveyspeed(fov[band], tobs(sensitivity, sefd[band], na, bw_max[band], eta[band]))   # total time in hours for given band, area in sq deg and sensitivity in Jy

# Effective survey speed for extragalactic specs (alpha = -0.7, gamma => euclidean).
# Effective survey speed for inverted spectrum (alpha = +0.7, gamma => euclidean).
ess_eg = lambda fov, dt, nu, sensitivity: ess_nuflux(fov, dt, nu, -0.7, sensitivity, -1.5)
ess_eg_norm = 9.e-15  # normalize to S band all sky survey
ess_inv = lambda fov, dt, nu, sensitivity: ess_nuflux(fov, dt, nu, 0.7, sensitivity, -1.5)
ess_inv_norm = 5e12    # normalize to Galactic Ku survey

survey_area = lambda dec_lim: (1+np.sin(np.radians(np.abs(dec_lim))))*(2*np.pi*(180/np.pi)**2)  # sq deg, exclude outside dec_lim


In [5]:
def find_hpnside(size):
    """ size in degrees
    """
    for i in range(20):
        resol = np.degrees(healpy.nside2resol(2**i))
        if resol < size:
            print(f'nside=2**{i} has resolution of {resol:1.1e} deg (resolves limit of {size} deg)')
            break
    return 2**i

## Calculate performance

In [6]:
print(f'Band \t   Tobs (s) \t    SS (OTF; deg2/hr) \t Sens. (Jy) \t hrs/3e4 deg2 (ideal)')
print('----------------------------------------------------------------------------------------------')
for band in ['DSA', 'VLAL', 'VLAS']:
    config = 'DFACS' if 'DSA' in band else 'VLASS'
    na0 = na[config]
    s0 = surveyspecs[config][1]
    t0 = tobs(s0, sefd[band], na0, bw_max[band], eta[band])
    s0 = sensitivity(sefd[band], t0, bw_max[band], eta[band], na0*(na0-1)/2)
    print(f'{band} \t   {t0:.1f}  \t\t {surveyspeed(fov[band], t0):.1f}   \t {s0:1.2e} \t {surveytime(band, s0, na0, 30000, 1.0):.1f} ')

Band 	   Tobs (s) 	    SS (OTF; deg2/hr) 	 Sens. (Jy) 	 hrs/3e4 deg2 (ideal)
----------------------------------------------------------------------------------------------
DSA 	   896.2  		 21.3   	 2.04e-06 	 1408.0 
VLAL 	   24.5  		 20.8   	 1.22e-04 	 1442.1 
VLAS 	   5.2  		 24.7   	 1.22e-04 	 1214.5 


## Re-calculate time required for each survey

In [7]:
def get_survey(surveyspecs):
    ss = 'Survey \t s_lim \t  SS \t Time/pointing \t Total Time\n'
    ss += '       \t (Jy)\t (deg/hr) \t (s)    \t   (hr)\n'
#    ss +='--------------------------------------------------------------------\n'
    for survey in sorted(surveyspecs.keys()):
        specs = surveyspecs[survey]
        area, s0, band, overhead, tsysfac = specs
        config = 'DFACS' if 'DSA' in band else 'VLASS'
        na0 = na[config]
        tint = tobs(s0, sefd[band], na0, bw_max[band], eta[band])
        t0_pointing = tobs(s0, sefd[band], na0, bw_max[band], eta[band])
        t0_survey = tsysfac*surveytime(band, s0, na0, area, overhead)
        surveyspeed0 = surveyspeed(fov[band], t0_pointing)
        ss += f'{survey} \t {s0:.2e} \t {surveyspeed0:.1f} \t     {t0_pointing:.1f} \t {t0_survey:.1f}\n'
    return ss

def print_survey(surveyspecs):
    ss = get_survey(surveyspecs)
    print(ss)

print_survey(surveyspecs)

Survey 	 s_lim 	  SS 	 Time/pointing 	 Total Time
       	 (Jy)	 (deg/hr) 	 (s)    	   (hr)
DFACS 	 2.04e-06 	 21.3 	     896.2 	 1760.0
VLASS 	 1.22e-04 	 24.7 	     5.2 	 1749.4



In [8]:
print(get_survey(surveyspecs))


Survey 	 s_lim 	  SS 	 Time/pointing 	 Total Time
       	 (Jy)	 (deg/hr) 	 (s)    	   (hr)
DFACS 	 2.04e-06 	 21.3 	     896.2 	 1760.0
VLASS 	 1.22e-04 	 24.7 	     5.2 	 1749.4



# healpix calcs of coverage and number of pixels

In [9]:
dec_min = -27
nside_thetap = find_hpnside(3.06)
n_pointings = int(np.round(healpy.nside2npix(nside_thetap)*(survey_area(dec_min)/survey_area(-90))))
thetap_area = healpy.nside2pixarea(nside_thetap, degrees=True)
print(f'{n_pointings} pointings with area {thetap_area:.2f} deg^2')

nside=2**5 has resolution of 1.8e+00 deg (resolves limit of 3.06 deg)
8933 pointings with area 3.36 deg^2


In [10]:
oversample = 4  # pixel/beam
nside_thetab = find_hpnside(resolution(1.35e9, 15e3)/3600/oversample)
n_pix_tot = int(np.round(healpy.nside2npix(nside_thetab)*(survey_area(dec_min)/survey_area(-90))))
thetab_resol = healpy.nside2resol(nside_thetab, arcmin=True)*60
print(f'{n_pix_tot/1024**4} Tpixels with resolution {thetab_resol:.2f} arcsec')
print(f'{8*n_pix_tot/1024**4} TB to pixelate whole sky (per product)')

print(f'{nside_thetab//nside_thetap} pixels per pointing (thetab side length)')
print(f'{int(2*1.19*nside_thetab//nside_thetap)} pixels per pointing (first null side length)')

nside=2**19 has resolution of 1.1e-04 deg (resolves limit of 0.00021220659078919376 deg)
2.1809857496091354 Tpixels with resolution 0.40 arcsec
17.447885996873083 TB to pixelate whole sky (per product)
16384 pixels per pointing (thetab side length)
38993 pixels per pointing (first null side length)


Open quesions:

- What is the exact wide survey sky area? Deep fields?
- What is an optimal cadence?
- What is the RC output image size, resolution, and bit depth?
- Are there constraints or processing implications for shorter total integration times?

Other surveys:
- Spectroscopy (DHIS) -- 9700 channels at 134kHz resolution, 4700 channels at 8.4kHz covering HI plus OH
- Pulsars/FRBs (DPuBS) -- 16384 channels, 0.8 ms sampling with 20 seconds triggers => 3 GB/beam/trigger and 1 s dedispersed voltage cutout per candidate, 2e9 samples/s => 64 TB/trigger 
- Polarimetry -- 1000 channels (1.3 MHz, Stokes QUV)